In [1]:
#JSP.ipynb

#讀檔
import pandas as pd
import numpy as np
import time
import pulp  

processing_time=pd.read_excel("Jobshop.xlsx",sheet_name="processing_time",header = None)
machine_order=pd.read_excel("Jobshop.xlsx",sheet_name="machine_order",header = None)

pt = processing_time.values
mo = machine_order.values

model = pulp.LpProblem("MIN_makespan", pulp.LpMinimize)

# 決策變數
dv = pulp.LpVariable.dicts("start_time",((i, j) for i in range(6) for j in range(6)),lowBound=0,cat='Continuous')

b =  pulp.LpVariable.dicts("binary_var",((i, j,o) for i in range(6) for j in range(6) for o in range(6) if j<o),lowBound=0,cat='Binary')

Cmax = pulp.LpVariable('Cmax',lowBound = 0, cat='Continuous')

# 目標式 : Min Cmax
model = pulp.LpProblem("MIN_makespan", pulp.LpMinimize)
model += Cmax

# 限制式:
for j in range(6):
    for i in range(5):
       I = mo[j,i]
       K = mo[j,i+1]
       J = j
       model += (dv[K,J] - dv[I,J]) >= pt[J][list(mo[J]).index(I)]
       
for j in range(6):
    for i in range(6):
       I = mo[j,i]
       J = j
       model += (Cmax - dv[I,J]) >= pt[J][list(mo[J]).index(I)]
       
for i in range(6):
    for j in range(6):
        for l in range(6):
            if j<l:
                I=i
                L = l
                J = j
                model += (dv[I,J] - dv[I,L]) >= (pt[L][list(mo[L]).index(I)] - 100*(1-b[i,J,L]))
                model += (dv[I,L] - dv[I,J]) >= (pt[J][list(mo[J]).index(I)] - 100*(b[i,J,L]))

# 求解       
start_time = time.time()
model.solve()
pulp.LpStatus[model.status]     

for var in dv:
    var_value = dv[var].varValue
    print( var[0],'-',var[1] ,var_value)

total_cost = pulp.value(model.objective)
print ('min cost:',total_cost)
print('the elapsed time:%s'% (time.time() - start_time))

0 - 0 6.0
0 - 1 39.0
0 - 2 20.0
0 - 3 15.0
0 - 4 49.0
0 - 5 29.0
1 - 0 25.0
1 - 1 0.0
1 - 2 31.0
1 - 3 9.0
1 - 4 22.0
1 - 5 14.0
2 - 0 5.0
2 - 1 8.0
2 - 2 0.0
2 - 3 22.0
2 - 4 13.0
2 - 5 49.0
3 - 0 32.0
3 - 1 49.0
3 - 2 8.0
3 - 3 27.0
3 - 4 54.0
3 - 5 17.0
4 - 0 49.0
4 - 1 15.0
4 - 2 38.0
4 - 3 30.0
4 - 4 25.0
4 - 5 45.0
5 - 0 39.0
5 - 1 29.0
5 - 2 12.0
5 - 3 46.0
5 - 4 42.0
5 - 5 20.0
min cost: 55.0
the elapsed time:1.0464897155761719


In [1]:
'''--------plot gantt chart-------'''
import pandas as pd
import plotly.plotly as py
import plotly.figure_factory as ff
import datetime

m_keys=[j for j in range(6)]
j_keys=[j for j in range(6)]
key_count={key:0 for key in j_keys}
j_count={key:0 for key in j_keys}
m_count={key:0 for key in m_keys}
j_record={}
#for i in sequence_best:
#    gen_t=int(pt[i][key_count[i]])
#    gen_m=int(ms[i][key_count[i]])
#    j_count[i]=j_count[i]+gen_t
#    m_count[gen_m]=m_count[gen_m]+gen_t
#    
#    if m_count[gen_m]<j_count[i]:
#        m_count[gen_m]=j_count[i]
#    elif m_count[gen_m]>j_count[i]:
#        j_count[i]=m_count[gen_m]
for var in dv:
    var_value = dv[var].varValue
    
    start_time=str(datetime.timedelta(seconds=var_value)) # convert seconds to hours, minutes and seconds
    end_time=str(datetime.timedelta(seconds=var_value+pt[var[1]][list(mo[var[1]]).index(var[0])]))
        
    j_record[(var[1],var[0])]=[start_time,end_time]
    
    key_count[i]=key_count[i]+1
        

df=[]
for m in m_keys:
    for j in j_keys:
        df.append(dict(Task='Machine %s'%(m), Start='2018-11-26 %s'%(str(j_record[(j,m)][0])), Finish='2018-11-26 %s'%(str(j_record[(j,m)][1])),Resource='Job %s'%(j)))
    
fig = ff.create_gantt(df, index_col='Resource', show_colorbar=True, group_tasks=True, showgrid_x=True, title='Job shop Schedule')
py.iplot(fig, filename='job_shop_scheduling', world_readable=True)

ImportError: 
The plotly.plotly module is deprecated,
please install the chart-studio package and use the
chart_studio.plotly module instead. 


In [2]:
pip install chart_studio.plotly

Note: you may need to restart the kernel to use updated packages.
  ERROR: Could not find a version that satisfies the requirement chart_studio.plotly (from versions: none)
ERROR: No matching distribution found for chart_studio.plotly
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
